# Cleans and formats the raw data

In [2]:
import pandas as pd
from datetime import datetime

data = pd.read_excel('Timeguessr_leaderboards_2.xlsx') 
data.to_csv('timeguessr.csv')
#data = pd.read_csv('timeguessr.csv', index_col=0)
# one column did not have a date, impute with guess
date_string = '2024-08-03 00:00:00'
data = data.rename(columns={'Unnamed: 36':  datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")})

columns = []
min_samples_to_keep = 1 # number of samples per day to keep.
for i in data.columns:
    if data[i].isna().sum() + min_samples_to_keep - 1 < len(data[i]): # skip days with to few observations
        columns.append(i)

new_format_date = datetime.strptime('2024-07-19 00:00:00', "%Y-%m-%d %H:%M:%S") # format changed after this date
for c in columns[1:]:
    if c > new_format_date:
        data[c] = data[c]*1_000

data = data[columns]
dataT = data.T
dataT.columns = dataT.iloc[0]
dataT.drop('Deltaker', inplace=True)
dates = [i.strftime('%m/%d') for i in dataT.T.columns] # make date format more readable

dataT.index = dates
dataT.values[dataT.values>50_000] = dataT.values[dataT.values>50_000]/1_000 # some inconsistent formating

# save data
dataT.to_csv('timeguessr_leaderboards_cleaned.csv')